In [ ]:
# install dependencies
!pip install -q --upgrade mido

     |████████████████████████████████| 51 kB 2.2 MB/s 


In [ ]:
from mido import MidiFile, MidiTrack, Message, MetaMessage
from enum import IntEnum
from collections import deque

In [ ]:
def tics_to_millis(tempo: int, ticks_per_beat: int, ticks: int) -> int:
  return int(round(tempo * ticks / (ticks_per_beat * 10000)))

class SpecialTokens(IntEnum):
  PADDING = 0
  START = 1
  END = 2

class Serializer:
  def __init__(self) -> None:
      self.max_power = 0
      self.NOTE_RANGE = 128
      self.VELOCITY_RANGE = 128
      self.SONG_SPLIT_THRESH = 300 # split songs if >= 3s space
      self.SONG_SKIP_THRESH = 1000 # skip song if <= 10s length
      self.time_shifts = []


  def encode_spaces(self, time_shift: int) -> list:
    self.time_shifts.append(time_shift)
    spaces = deque()
    mask = 1
    power = 0
    start = len(SpecialTokens) + self.NOTE_RANGE * 2# + self.COMPOSER_COUNT

    while time_shift >= mask:
      if time_shift & mask:
        self.max_power = max(power, self.max_power)
        spaces.appendleft(start + power)
      
      mask <<= 1
      power += 1
    
    return list(spaces)


  def tokenize(self, midi: MidiFile, do_not_skip=False) -> list:
    notes_on = {}
    for i in range(self.NOTE_RANGE):
      notes_on[i] = False

    out = [[SpecialTokens.START.value]]

    ppq = midi.ticks_per_beat
    tempo = midi.tracks[0][0].tempo
    time_shift = 0
    time_shift_global = 0
    for i, message in enumerate(midi.tracks[1]):
      if message.type == "track_name" or message.type == "program_change":
        continue
      
      time_shift += tics_to_millis(tempo, ppq, message.time)
      if message.type == "control_change":
        continue

      # skip initial pause
      if out[-1][-1] == SpecialTokens.START.value:
        time_shift = 0
      
      # split songs
      if time_shift >= self.SONG_SPLIT_THRESH:
        split = True
        # ensure no note was pressed
        for on in notes_on.values():
          if on:
            split = False
            break
        # split should happen
        if split:
          if time_shift_global <= self.SONG_SKIP_THRESH and not do_not_skip:
            # discard previous song if <= 10s
            out[-1] = [SpecialTokens.START.value]
          else:
            out[-1].append(SpecialTokens.END.value)
            out.append([SpecialTokens.START.value])
          time_shift = 0
          time_shift_global = 0

      # encode spaces
      if time_shift > 0:
        out[-1].extend(self.encode_spaces(time_shift))
        time_shift_global += time_shift
        time_shift = 0

      # encode note
      if message.type == "note_on":
        val = len(SpecialTokens) + message.note
        if message.velocity == 0:
          val += self.NOTE_RANGE
          notes_on[message.note] = False
        else:
          notes_on[message.note] = True
        out[-1].append(val)
    out[-1].append(SpecialTokens.END.value)

    # remove last song if too short
    if time_shift_global <= self.SONG_SKIP_THRESH and not do_not_skip:
      out = out[:-1]
    
    clean = []
    # trim final space
    for tokenized in out:
      index = -2
      while tokenized[index] >= self.__space_start():
        index -= 1
      clean.append(tokenized[:index + 1])
      clean[-1].append(SpecialTokens.END.value)
    return clean

  def __space_start(self):
    return len(SpecialTokens) + self.NOTE_RANGE * 2

  def vocab_size(self) -> int:
    return len(SpecialTokens) + self.NOTE_RANGE * 2 + 1 + self.max_power

  def deserialize(self, tokenized: list) -> MidiFile:
    mid = MidiFile(ticks_per_beat=500)
    ctrl_track = MidiTrack()
    ctrl_track.append(MetaMessage('set_tempo', tempo=500000, time=0))
    ctrl_track.append(MetaMessage('end_of_track', time=1))
    mid.tracks.append(ctrl_track)

    track = MidiTrack()
    time_shift = 0
    for token in tokenized:
      # skip padding
      if token == SpecialTokens.PADDING.value:
        continue

      # space
      if token >= self.__space_start():
        power = token - len(SpecialTokens) - self.NOTE_RANGE * 2
        time_shift += 1 << power
        continue
      
      if token == SpecialTokens.START.value:
        message = Message('program_change', channel=0, program=0)

      elif token == SpecialTokens.END.value:
        message = MetaMessage('end_of_track')
      else:
        # note
        shifted = token - len(SpecialTokens)
        velocity = 64
        note = shifted % self.NOTE_RANGE
        if shifted >= self.NOTE_RANGE:
          velocity = 0

        message = Message('note_on', channel=0, note=note, velocity=velocity)

      message.time = time_shift * 10
      time_shift = 0
      track.append(message)
    mid.tracks.append(track)
    return mid